In [3]:
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as pyplot

from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Utils.DataReader import load_urm, load_icm, load_target

Cloning into 'Recommender-Systems-Challenge-2022'...
remote: Enumerating objects: 619, done.
remote: Counting objects: 100% (215/215), done.
remote: Compressing objects: 100% (162/162), done.
remote: Total 619 (delta 107), reused 139 (delta 52), pack-reused 404
Receiving objects: 100% (619/619), 106.18 MiB | 11.85 MiB/s, done.
Resolving deltas: 100% (279/279), done.
Updating files: 100% (254/254), done.


In [4]:
URM_all = load_urm()

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.85)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 1536 ( 3.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 959 ( 2.3%) Users that have less than 1 test interactions


In [5]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

recommender_SLIMElasticNet = SLIMElasticNetRecommender(URM_all)
recommender_SLIMElasticNet.fit(topK=585, l1_ratio=0.009929794168572358, alpha=0.0018687691383859428) 

SLIMElasticNetRecommender: Processed 2559 (10.4%) in 5.00 min. Items per second: 8.53
SLIMElasticNetRecommender: Processed 5211 (21.3%) in 10.00 min. Items per second: 8.68
SLIMElasticNetRecommender: Processed 7957 (32.5%) in 15.00 min. Items per second: 8.84
SLIMElasticNetRecommender: Processed 10735 (43.8%) in 20.01 min. Items per second: 8.94
SLIMElasticNetRecommender: Processed 13681 (55.8%) in 25.01 min. Items per second: 9.12
SLIMElasticNetRecommender: Processed 16730 (68.3%) in 30.01 min. Items per second: 9.29
SLIMElasticNetRecommender: Processed 19623 (80.1%) in 35.01 min. Items per second: 9.34
SLIMElasticNetRecommender: Processed 22502 (91.8%) in 40.01 min. Items per second: 9.37
SLIMElasticNetRecommender: Processed 24507 (100.0%) in 43.31 min. Items per second: 9.43


In [6]:
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

recommender_EaseR = EASE_R_Recommender(URM_all)
recommender_EaseR.fit(topK=None, normalize_matrix=False, l2_norm=157.86876317814773)

EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 10.66 min


In [7]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

recommender_RP3beta = RP3betaRecommender(URM_all)
recommender_RP3beta.fit(alpha=0.6949339074999242, beta=0.2853519331891143, topK=59, implicit=True)

RP3betaRecommender: Similarity column 24507 (100.0%), 1067.58 column/sec. Elapsed time 22.96 sec


In [20]:
from numpy import linalg as LA
from Recommenders.BaseRecommender import BaseRecommender

class DifferentLossScoresMultiHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of three predictions scores
    R = R1*alpha + R2*beta + R3*(1-alpha-beta)
    
    Class from Dacrema exercise modified by Antonio Ercolani
    The original took as input 2 recommender

    """

    RECOMMENDER_NAME = "DifferentLossScoresMultiHybridRecommender"


    def __init__(self, URM_train, recommender_1, recommender_2, recommender_3):
        super(DifferentLossScoresMultiHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        self.recommender_3 = recommender_3
        
        
        
    def fit(self, norm, alpha = 0.5, beta = 0.5):

        self.alpha = alpha
        self.beta = beta
        self.norm = norm


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array, items_to_compute = None)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array, items_to_compute = None)
        item_weights_3 = self.recommender_3._compute_item_score(user_id_array, items_to_compute = None)

        norm_item_weights_1 = LA.norm(item_weights_1, self.norm)
        norm_item_weights_2 = LA.norm(item_weights_2, self.norm)
        norm_item_weights_3 = LA.norm(item_weights_3, self.norm)
        
        
        if norm_item_weights_1 == 0:
            raise ValueError("Norm {} of item weights for recommender 1 is zero. Avoiding division by zero".format(self.norm))
        
        if norm_item_weights_2 == 0:
            raise ValueError("Norm {} of item weights for recommender 2 is zero. Avoiding division by zero".format(self.norm))
            
        if norm_item_weights_3 == 0:
            raise ValueError("Norm {} of item weights for recommender 3 is zero. Avoiding division by zero".format(self.norm))
        
        item_weights = item_weights_1 / norm_item_weights_1 * self.alpha + item_weights_2 / norm_item_weights_2 * self.beta + item_weights_3 / norm_item_weights_3 * (1-self.alpha-self.beta)

        return item_weights

In [24]:
recommender_multi = DifferentLossScoresMultiHybridRecommender(URM_all, recommender_SLIMElasticNet, recommender_EaseR, recommender_RP3beta)
recommender_multi.fit(norm=1, alpha = 0.55, beta = 0.35)

In [19]:
from numpy import linalg as LA
from Recommenders.BaseRecommender import BaseRecommender

class DifferentLossScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1/norm*alpha + R2/norm*(1-alpha) where R1 and R2 come from
    algorithms trained on different loss functions.

    """

    RECOMMENDER_NAME = "DifferentLossScoresHybridRecommender"


    def __init__(self, URM_train, recommender_1, recommender_2):
        super(DifferentLossScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        
        
        
    def fit(self, norm, alpha = 0.5):

        self.alpha = alpha
        self.norm = norm


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array, items_to_compute = None)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array, items_to_compute = None)

        norm_item_weights_1 = LA.norm(item_weights_1, self.norm)
        norm_item_weights_2 = LA.norm(item_weights_2, self.norm)
        
        
        if norm_item_weights_1 == 0:
            raise ValueError("Norm {} of item weights for recommender 1 is zero. Avoiding division by zero".format(self.norm))
        
        if norm_item_weights_2 == 0:
            raise ValueError("Norm {} of item weights for recommender 2 is zero. Avoiding division by zero".format(self.norm))
        
        item_weights = item_weights_1 / norm_item_weights_1 * self.alpha + item_weights_2 / norm_item_weights_2 * (1-self.alpha)

        return item_weights

In [26]:
recommender_duo = DifferentLossScoresHybridRecommender(URM_all, recommender_SLIMElasticNet, recommender_EaseR)
recommender_duo.fit(norm = 1, alpha = 0.4)

In [22]:
from numpy import linalg as LA
from Recommenders.BaseRecommender import BaseRecommender

class MergeRaccomandationsHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1/norm*alpha + R2/norm*(1-alpha) where R1 and R2 come from
    algorithms trained on different loss functions.

    """

    RECOMMENDER_NAME = "MergeRaccomandationsHybridRecommender"


    def __init__(self, URM_train, recommender_1, recommender_2):
        super(MergeRaccomandationsHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
    
    def _compute_item_score(self, user_id_array, items_to_compute):
        
        # In a simple extension this could be a loop over a list of pretrained recommender objects
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array, items_to_compute = None)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array, items_to_compute = None)

        item_weights = item_weights_1*0.5 + item_weights_2*0.5

        return item_weights
        
    def recommend(self, user_id_array, cutoff = None, remove_seen_flag=True, items_to_compute = None,
                  remove_top_pop_flag = False, remove_custom_items_flag = False, return_scores = False):

        # If is a scalar transform it in a 1-cell array
        if np.isscalar(user_id_array):
            user_id_array = np.atleast_1d(user_id_array)
            single_user = True
        else:
            single_user = False

        if cutoff is None:
            cutoff = self.URM_train.shape[1] - 1

        cutoff = min(cutoff, self.URM_train.shape[1] - 1)

        # Compute the scores using the model-specific function
        # Vectorize over all users in user_id_array
        scores_batch = self._compute_item_score(user_id_array, items_to_compute=items_to_compute)


        for user_index in range(len(user_id_array)):

            user_id = user_id_array[user_index]

            if remove_seen_flag:
                scores_batch[user_index,:] = self._remove_seen_on_scores(user_id, scores_batch[user_index, :])

            # Sorting is done in three steps. Faster then plain np.argsort for higher number of items
            # - Partition the data to extract the set of relevant items
            # - Sort only the relevant items
            # - Get the original item index
            # relevant_items_partition = (-scores_user).argpartition(cutoff)[0:cutoff]
            # relevant_items_partition_sorting = np.argsort(-scores_user[relevant_items_partition])
            # ranking = relevant_items_partition[relevant_items_partition_sorting]
            #
            # ranking_list.append(ranking)


        if remove_top_pop_flag:
            scores_batch = self._remove_TopPop_on_scores(scores_batch)

        if remove_custom_items_flag:
            scores_batch = self._remove_custom_items_on_scores(scores_batch)

        # relevant_items_partition is block_size x cutoff
        relevant_items_partition = (-scores_batch).argpartition(cutoff, axis=1)[:,0:cutoff]

        # Get original value and sort it
        # [:, None] adds 1 dimension to the array, from (block_size,) to (block_size,1)
        # This is done to correctly get scores_batch value as [row, relevant_items_partition[row,:]]
        relevant_items_partition_original_value = scores_batch[np.arange(scores_batch.shape[0])[:, None], relevant_items_partition]
        relevant_items_partition_sorting = np.argsort(-relevant_items_partition_original_value, axis=1)
        ranking = relevant_items_partition[np.arange(relevant_items_partition.shape[0])[:, None], relevant_items_partition_sorting]
        
        ranking_list = [None] * ranking.shape[0]

        # Remove from the recommendation list any item that has a -inf score
        # Since -inf is a flag to indicate an item to remove
        for user_index in range(len(user_id_array)):
            user_recommendation_list = ranking[user_index]
            user_item_scores = scores_batch[user_index, user_recommendation_list]

            not_inf_scores_mask = np.logical_not(np.isinf(user_item_scores))

            user_recommendation_list = user_recommendation_list[not_inf_scores_mask]
            ranking_list[user_index] = user_recommendation_list.tolist()
            
            
        # MODIFIED PART TO MERGE THE LISTS
        list1 = self.recommender_1.recommend(user_id_array, cutoff = cutoff)
        list2 = self.recommender_2.recommend(user_id_array, cutoff = cutoff)
        
        list1 = list1[0]
        list2 = list2[0]
        
        result = []
        i = 0
            
        while len(result) < cutoff:
            if list1[i] not in result:
                result.append(list1[i])
            if (list2[i] != list1[i]):
                if list2[i] not in result:
                    if len(result) < cutoff:
                        result.append(list2[i])
            i = i + 1
        
        if return_scores:
            return result, scores_batch
        else:
            return result

In [25]:
recommender_multi.recommend(user_id_array=0, cutoff = 10)

[617, 752, 18484, 391, 20, 2488, 23, 673, 56, 107]

In [27]:
recommender_duo.recommend(user_id_array=0, cutoff = 10)

[752, 617, 20, 391, 18484, 673, 2488, 23, 56, 29]

In [28]:
recommender = MergeRaccomandationsHybridRecommender(URM_all, recommender_multi, recommender_duo)
recommender.recommend(user_id_array=0, cutoff = 10)

[617, 752, 18484, 20, 391, 2488, 673, 23, 56, 107]

## Submissions

In [29]:
test_users = pd.read_csv('Dataset/data_target_users_test.csv')

In [32]:
user_id = test_users['user_id']
recommendations = []
for user in user_id:
    recommendations.append(recommender_multi.recommend(user, cutoff=10))

In [33]:
for index in range(len(recommendations)):
    recommendations[index]=np.array(recommendations[index])

test_users['item_list']= recommendations
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])
test_users.to_csv('Submissions/Submission_Hybrid_ListMerge.csv', index=False)